# 实验二  利用K-L变换法进行人脸识别
主分量分析是一种有效的通过线性变换来降低特征维数的方法，也称为Karhunen-Loeve变换（K-L变换）。此变换是一种基于目标统计特性的最佳正交变换，它的最佳性体现在变换后产生的新的分量正交或不相关。

编写用K-L变换法进行人脸识别的程序，对标准图像库进行人脸识别实验。
ORL_faces图像库中有40组人脸图像，每组内的10幅图像取自同一个人，部分人脸图像如下图所示。利用每组中的前5幅人脸图像（共200幅）作为训练样本，生成人脸识别的分类器，显示“平均脸” 图像和“特征脸” 图像。用剩下的200幅图像作为测试样本，输出识别结果，检测识别率。


## 代码& 结果
### 定义PCA类

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline
from PIL import Image

class PCA:
    def __init__(self,n_components=None,enegy=None):
        '''
            初始化时，n_components 和 enegy 必须有一个，同时存在时忽略enegy
        '''
        self.enegy=enegy
        self.n_components=n_components
        self.mean=None
        self.eig_vector=None
        self.final_index=None

    def fit(self,X):
        '''
            传入 X 维度为 (N*p), N为样本数，p为维度
            fit 仅训练样本
        '''
        def Mean(X):
            return X.mean(axis=0)
        def Cov(X):
            return (X-Mean(X)).T.dot(X-Mean(X))/(X.shape[0])
            return np.cov(X.T)
        def Eig(X):
            return np.linalg.eig(X)

        self.mean=Mean(X)
        centralized=X-self.mean
        cov=Cov(centralized)
        eig_value, eig_vector=Eig(cov)
        self.eig_vector=eig_vector

        sort_eig_index=np.argsort(-eig_value)
        if self.n_components!=None:                                         ##如果指定选择前n个分量
            self.final_index=sort_eig_index[:self.n_components]
        elif self.enegy!=None:                                              ##如果没有指定前n个分量，且指定保留能量比例
            now_enegy=0.
            tot_enegy=eig_value.sum()
            for i in range(eig_vector.shape[0]):
                if now_enegy/tot_enegy>self.enegy:
                    break
                now_enegy+=eig_value[sort_eig_index[i]]
            self.final_index=sort_eig_index[:i]

        print('保留的主成分: '+str(self.final_index.shape[0]))

    def transform(self,X):
        '''
            传入 X 维度为 (N*p), N为样本数，p为维度
            返回转换后的数据，维度为 (N*k), k是保留的主成分数
        '''
        assert X.shape[1]==self.eig_vector.shape[1]
        centralized=X-self.mean
        new_components=self.eig_vector[:,self.final_index].T.dot(centralized.T).T
        return new_components
    
    def inverse_transform(self,X_pca):
        assert X_pca.shape[1]==self.component_.shape[0]
        pass
        return X_pca.dot(self.component_)


#A = np.array([[1, 2,10,100,100], [2, 3,20,200,200], [3, 4,30,300,300],[4,5,40,400,400],[4,5,40,400,400]])
#print('A shape :'+str(A.shape))
#pca=PCA(enegy=0.99)
#pca.fit(A)
#pca.transform(A)


### 读取数据& 训练

In [39]:
## 读取训练图片及测试图片,reshape为输入格式（N，P）
y_train=[]
x_train=[]
for i in range(40):
    for j in range(1,6):
        im=Image.open('./faces/{}/{}.pgm'.format(i,j))
        im=np.array(im)
        y_train.append(i)
        x_train.append(im.reshape((-1)))
x_train=np.array(x_train,dtype=np.float64) 

y_test=[]
x_test=[]
for i in range(40):
    for j in range(6,11):
        im=Image.open('./faces/{}/{}.pgm'.format(i,j))
        im=np.array(im)
        y_test.append(i)
        x_test.append(im.reshape((-1)))
x_test=np.array(x_test,dtype=np.float64)

## 训练以及映射
pca=PCA(enegy=0.99)
pca.fit(x_train)
rebuild_x_train=pca.transform(x_train)
rebuild_x_test=pca.transform(x_test)



保留的主成分: 170


### 准确率&结果展示
第一行为样本0，第二行为样本1，以此类推

In [43]:
ans=[]
for i in range(rebuild_x_test.shape[0]):
    dist=[]
    for j in range(rebuild_x_train.shape[0]):
        dist.append(np.sqrt(np.sum(np.square(rebuild_x_test[i] - rebuild_x_train[j])))) #使用欧拉距离
    dist=np.array(dist)
    ans.append(np.argmin(dist)//5)
ans=np.array(ans)

correct=(ans==y_test).sum()
acc=correct/ans.shape[0]
print('准确率 ：'+str(acc))
ans.reshape(-1,5)

准确率 ：0.895


array([[ 5,  0,  0,  0,  5],
       [ 1,  1,  1,  1,  1],
       [ 2,  2,  2,  2,  2],
       [ 3,  3,  3,  3,  3],
       [ 4,  4,  4,  4,  4],
       [ 5,  5,  5,  5,  0],
       [ 6,  6,  6,  6,  6],
       [ 7,  7,  7,  7,  7],
       [ 8,  8,  8,  8,  8],
       [ 9,  9,  9,  9,  9],
       [10, 10, 10, 10, 38],
       [11, 11, 15, 11, 11],
       [12, 12, 12, 12, 12],
       [13, 13, 13, 13, 13],
       [14, 14, 14, 22, 14],
       [15, 15, 15, 15, 15],
       [16, 16,  1, 16, 16],
       [36, 24, 17, 17, 36],
       [18, 18, 18, 18, 18],
       [19, 19, 19, 15, 19],
       [20, 20, 38, 20, 20],
       [21, 21, 21, 21, 21],
       [22, 22, 22, 22, 22],
       [23, 23, 23, 23, 23],
       [24, 24, 24, 24, 24],
       [25, 25, 25, 25, 25],
       [26, 26, 26, 26, 26],
       [17, 17, 17, 27, 27],
       [28, 28, 37, 28, 28],
       [29, 29, 29, 29, 29],
       [30, 30, 30, 30, 30],
       [31, 30, 31, 31, 31],
       [32,  2, 32, 32, 32],
       [33, 33, 33, 33, 33],
       [34, 34